In [2]:
%load_ext autoreload
%autoreload 2

import os
import json
import copy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
import ray_results_interpreter as rri
import subprocess
import concurrent.futures
from main_run import MainRun

/user/ml4723/.conda/envs/neural_inventory_control/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/user/ml4723/.conda/envs/neural_inventory_control/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
def collect_data_and_plot(df, test_mode, setting_name, underage_cost):
    def run_main_run(row):
        try:
            cmd = [
                "/user/ml4723/.conda/envs/neural_inventory_control/bin/python",
                "main_run.py",
                test_mode, # test or test_on_dev
                setting_name,
                row['hyperparam_name'],
                row['path'],
                row['Architecture Class']
            ]
            env = {
                **os.environ,
                "MKL_THREADING_LAYER": "GNU",
                "MKL_SERVICE_FORCE_INTEL": "1"
            }
            subprocess.run(cmd, capture_output=True, text=True, check=True, env=env, cwd="/user/ml4723/Prj/NIC/")
        except subprocess.CalledProcessError as e:
            print(f"Error running main_run.py for path {row['path']}: {e}")
            print(f"Error output: {e.stderr}")
        except Exception as e:
            print(f"Unexpected error running main_run.py for path {row['path']}: {e}")

    def get_file_name(row):
        return f"results/one_warehouse_real/{row['# of stores']}/{row['Architecture Class']}/{underage_cost}.csv"
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        futures = []
        for _, row in df.iterrows():
            file_name = get_file_name(row)
            if os.path.exists(file_name):
                continue
            while len(futures) >= 24:
                done, not_done = concurrent.futures.wait(
                    futures, 
                    return_when=concurrent.futures.FIRST_COMPLETED
                )
                futures = list(not_done)
            futures.append(executor.submit(run_main_run, row))
        concurrent.futures.wait(futures)
    
    avg_s_underage = []
    avg_s_holding = []
    avg_w_holding = []
    avg_cost = []
    for _, row in df.iterrows():
        file_name = get_file_name(row)
        data = pd.read_csv(file_name)
        n_samples = 32768
        batch_size = 4096
        num_batches = n_samples // batch_size
        num_steps = 500
        start_step = 300
        
        relevant_indices = []
        for batch in range(num_batches):
            batch_start = batch * batch_size * num_steps
            step_start = batch_start + start_step * batch_size
            step_end = batch_start + num_steps * batch_size
            relevant_indices.extend(range(step_start, step_end))
        relevant_data = data.iloc[relevant_indices]
        
        n_stores = row['# of stores']
        avg_s_underage.append(relevant_data['s_underage_costs'].mean() / underage_cost)
        avg_s_holding.append(relevant_data['s_holding_costs'].mean() / underage_cost)
        avg_w_holding.append(relevant_data['w_holding_costs'].mean() / underage_cost)
        avg_cost.append(relevant_data['s_underage_costs'].sum() * n_stores + relevant_data['s_holding_costs'].sum() * n_stores + relevant_data['w_holding_costs'].sum())

    # Create a copy of the dataframe to avoid SettingWithCopyWarning
    df_copy = df.copy()
    df_copy['avg_s_underage'] = avg_s_underage
    df_copy['avg_s_holding'] = avg_s_holding
    df_copy['avg_w_holding'] = avg_w_holding 
    df_copy['avg_cost'] = avg_cost
    df = df_copy

    # Calculate relative cost
    # Calculate cost as percentage of lowest cost
    # Calculate min cost for each number of stores
    df['cost'] = df.groupby('# of stores')['avg_cost'].transform(lambda x: x/x.min() * 100)

    plot_data = df.pivot(index="# of stores", columns='Architecture Class', 
                         values=['cost', 'avg_s_underage', 'avg_s_holding', 'avg_w_holding'])
    
    fig, axes = plt.subplots(2, 2, figsize=(20, 12))
    axes = axes.flatten()  # Flatten the 2D array of axes to 1D for easier iteration
    titles = ['Cost (% of minimum at each # of stores)', 'Average Store lost sales / (average unit underage cost)', 'Average Store Holding Cost / (average unit underage cost)', 'Average Warehouse Holding Cost / (average unit underage cost)']
    y_values = ['cost', 'avg_s_underage', 'avg_s_holding', 'avg_w_holding']

    x_values = df['# of stores'].unique()
    for i, (ax, title, y_value) in enumerate(zip(axes, titles, y_values)):
        for arch in df['Architecture Class'].unique():
            if arch == 'Just_In_Time':
                continue
            if arch in plot_data[y_value].columns:
                ax.plot(x_values, plot_data[y_value][arch].loc[x_values], marker='o', label=f'{arch}', 
                        color=color_scheme[arch], linestyle=linestyle_scheme[arch])
        
        ax.set_xlabel('Number of stores')
        ax.set_ylabel(f'{title}')
        ax.set_title(title)
        ax.legend()
        ax.grid(True)
        ax.set_xticks(x_values)
        ax.set_xticklabels(x_values)
    
    plt.tight_layout()
    plt.show()
    return df


In [4]:
# Define color scheme for architecture types
color_scheme = {
    "GNN_MP_transshipment": "#ff7f0e",  # Orange
    "GNN_MP_NN_Per_Layer_transshipment": "#2ca02c",  # Green
    "vanilla_transshipment": "#9467bd",  # Purple
}

linestyle_scheme = {
    "GNN_MP_transshipment": "-",  # Solid
    "GNN_MP_NN_Per_Layer_transshipment": "-",  # Solid
    "vanilla_transshipment": "-",  # Solid
}

testset_name = "generic_architecture_transshipment"

# Define paths for each architecture
architectures = {
    "vanilla_transshipment": {},
    "GNN_MP_transshipment": {},
    "GNN_MP_NN_Per_Layer_transshipment": {}
}

for arch in architectures:
    architectures[arch] = {
        n_stores: f'/user/ml4723/Prj/NIC/ray_results/{testset_name}/{arch}/{n_stores}'
        for n_stores in [3, 5, 10]
    }

def custom_data_filler(out_row, reference_row):
    out_row['path'] = reference_row['path']

def default_condition_setter(condition_name):
    return None

sort_by = 'test_loss'
pick_row_from_run_by = 'test_loss'

results_interpretor = rri.RayResultsinterpreter()

# Create dataframes for each architecture and parameter combination
dfs = []
params = {
    'store_lead_time': [2, 6],
    'store_underage_cost': [4, 9],
    'stores_correlation': [0.0, 0.5]
}

for arch_name, paths in architectures.items():
    for lead_time in params['store_lead_time']:
        for underage_cost in params['store_underage_cost']:
            for correlation in params['stores_correlation']:
                df = results_interpretor.make_table(paths,
                    {'store_lead_time': lead_time,
                     'store_underage_cost': underage_cost,
                     'stores_correlation': correlation},
                    default_condition_setter, custom_data_filler, 
                    sort_by=sort_by, pick_row_from_run_by=pick_row_from_run_by)
                
                df.insert(2, 'Architecture Class', arch_name)
                df.insert(1, 'hyperparam_name', arch_name)
                df['store_lead_time'] = lead_time
                df['store_underage_cost'] = underage_cost
                df['stores_correlation'] = correlation
                dfs.append(df)

# Combine all dataframes
df = pd.concat(dfs, ignore_index=True)

# Calculate test gap percentage
min_test_loss = df.groupby(['# of stores', 'store_lead_time', 'store_underage_cost', 'stores_correlation'])['Test Loss'].transform('min')
df['Test Gap %'] = ((df['Test Loss'] - min_test_loss) / min_test_loss) * 100

# Create 6 subplots (2x3)
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
axes = axes.flatten()

# Plot for each store_lead_time
for i, lead_time in enumerate(params['store_lead_time']):
    data = df[df['store_lead_time'] == lead_time]
    pivot_df = data.groupby(['# of stores', 'Architecture Class'])['Test Gap %'].mean().unstack()
    
    ax = axes[i]
    for arch in pivot_df.columns:
        ax.plot(pivot_df.index, pivot_df[arch], 
                marker='o', label=arch,
                color=color_scheme[arch],
                linestyle=linestyle_scheme[arch])
    ax.set_title(f'Store Lead Time = {lead_time}')
    ax.set_xlabel('Number of Stores')
    ax.set_ylabel('Test Gap %')
    ax.grid(True)
    ax.legend()

# Plot for each store_underage_cost
for i, underage_cost in enumerate(params['store_underage_cost']):
    data = df[df['store_underage_cost'] == underage_cost]
    pivot_df = data.groupby(['# of stores', 'Architecture Class'])['Test Gap %'].mean().unstack()
    
    ax = axes[i+2]
    for arch in pivot_df.columns:
        ax.plot(pivot_df.index, pivot_df[arch],
                marker='o', label=arch,
                color=color_scheme[arch],
                linestyle=linestyle_scheme[arch])
    ax.set_title(f'Store Underage Cost = {underage_cost}')
    ax.set_xlabel('Number of Stores')
    ax.set_ylabel('Test Gap %')
    ax.grid(True)
    ax.legend()

# Plot for each stores_correlation
for i, correlation in enumerate(params['stores_correlation']):
    data = df[df['stores_correlation'] == correlation]
    pivot_df = data.groupby(['# of stores', 'Architecture Class'])['Test Gap %'].mean().unstack()
    
    ax = axes[i+4]
    for arch in pivot_df.columns:
        ax.plot(pivot_df.index, pivot_df[arch],
                marker='o', label=arch,
                color=color_scheme[arch],
                linestyle=linestyle_scheme[arch])
    ax.set_title(f'Store Correlation = {correlation}')
    ax.set_xlabel('Number of Stores')
    ax.set_ylabel('Test Gap %')
    ax.grid(True)
    ax.legend()

plt.tight_layout()
plt.show()

Error processing files in /user/ml4723/Prj/NIC/ray_results/generic_architecture_transshipment/vanilla_transshipment/3/run_2024-12-29_14-32-04/run_95668_00003_3_config=transshipment_backlogged,early_stop_check_epochs=25,samples=1,stop_if_no_improve_for_epochs=100,store_lea_2024-12-29_14-32-04: 'test_loss'
Error processing files in /user/ml4723/Prj/NIC/ray_results/generic_architecture_transshipment/vanilla_transshipment/3/run_2024-12-29_14-32-04/run_95668_00002_2_config=transshipment_backlogged,early_stop_check_epochs=25,samples=3,stop_if_no_improve_for_epochs=100,store_lea_2024-12-29_14-32-04: 'test_loss'
Error processing files in /user/ml4723/Prj/NIC/ray_results/generic_architecture_transshipment/vanilla_transshipment/3/run_2024-12-29_14-32-04/run_95668_00001_1_config=transshipment_backlogged,early_stop_check_epochs=25,samples=2,stop_if_no_improve_for_epochs=100,store_lea_2024-12-29_14-32-04: 'test_loss'
Error processing files in /user/ml4723/Prj/NIC/ray_results/generic_architecture_tr

IndexError: index 2 is out of bounds for axis 0 with size 0

In [17]:
df_t = collect_data_and_plot(df, "test", "one_warehouse_lost_demand_exp_underage_cost_random_yield", 1)